### This is what Silvia and Simon tried in order to read the data provided as example on the Prithvi HuggingFace demo and to get some Sentinel data and make it look the same.

### The main takeaway is that the data should be Sentinel-2 **L1C** data. Which can be downloaded completely with all the expected layers from Copernicus using the sentinelhub package.


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

# read data
spain = rasterio.open('data/Spain_7370579_S2Hand.tif')
show(spain)


In [ ]:
spain = rasterio.open('data/Spain_7370579_LabelHand.tif')
show(spain)


In [ ]:
# Open the image file

    # Read the image bands
bands = spain.read()

# Display each band
for band_index in range(spain.count):
    band = bands[band_index]
    band.shape
    plt.imshow(bands[band_index])  # You can specify other colormaps as needed
    plt.title(f"Band {band_index + 1}")
    plt.show()


In [ ]:
# Combine the bands from data/B*.tif into a single image
target_bands = [
    "B02",
    "B03",
    "B04",
    "B8A",
    "B11",
    "B12",
]

datasets = []

for target_band in target_bands:
    dataset = rasterio.open(f"data/{target_band}.tif")
    band = dataset.read(1)
    band.shape
    datasets.append(dataset)
    # plt.imshow(band, cmap='gray')
    # plt.title(target_band)
    # plt.show()

# new_dataset = rasterio.open(
#     'data/new.tif',
#     'w',
#     driver='GTiff',
#     height=Z.shape[0],
#     width=Z.shape[1],
#     count=1,
#     dtype=Z.dtype,
#     crs='+proj=latlong',
#     transform=transform,
# )


In [ ]:
from rasterio.merge import merge

mosaic, output = merge(datasets)


In [ ]:
output_meta = dataset.meta.copy()
output_meta.update(
   {"driver": "GTiff",
       "height": mosaic.shape[1],
       "width": mosaic.shape[2],
       "transform": output,
   })


In [ ]:
show(mosaic)


In [ ]:
response = rasterio.open("data/6010d416b76925d6ff045c57962858c8/response.tiff")
response.meta


In [ ]:
with rasterio.open("data/new.tif", "w", **output_meta) as m:
   m.write(mosaic)
